<a href="https://colab.research.google.com/github/atpathak/DeepLearning_code-templates_2026/blob/main/AutoEncoder_RecommendationSystem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
!unzip Archive.zip

Archive:  Archive.zip
   creating: ml-1m/
  inflating: __MACOSX/._ml-1m        
  inflating: ml-1m/.Rhistory         
  inflating: __MACOSX/ml-1m/._.Rhistory  
  inflating: ml-1m/.DS_Store         
  inflating: __MACOSX/ml-1m/._.DS_Store  
  inflating: ml-1m/movies.dat        
  inflating: __MACOSX/ml-1m/._movies.dat  
  inflating: ml-1m/Train_Test_Set_Creation.R  
  inflating: __MACOSX/ml-1m/._Train_Test_Set_Creation.R  
  inflating: ml-1m/test_set.csv      
  inflating: __MACOSX/ml-1m/._test_set.csv  
  inflating: ml-1m/ratings.dat       
  inflating: __MACOSX/ml-1m/._ratings.dat  
  inflating: ml-1m/README            
  inflating: __MACOSX/ml-1m/._README  
  inflating: ml-1m/ratings.csv       
  inflating: __MACOSX/ml-1m/._ratings.csv  
  inflating: ml-1m/training_set.csv  
  inflating: __MACOSX/ml-1m/._training_set.csv  
  inflating: ml-1m/users.dat         
  inflating: __MACOSX/ml-1m/._users.dat  
   creating: ml-100k/
  inflating: __MACOSX/._ml-100k      
  inflating: ml-100k/u.

In [3]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [6]:
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [7]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter= '\t')
training_set = np.array(training_set, dtype = 'int')

test_set = pd.read_csv('ml-100k/u1.test', delimiter= '\t')
test_set = np.array(test_set, dtype = 'int')

In [10]:
test_set.shape, training_set.shape

((19999, 4), (79999, 4))

In [11]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1][data[:,0] == id_users]
    id_ratings = data[:, 2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [12]:
training_set = convert(training_set)
test_set = convert(test_set)

In [19]:
training_set[0][:5]

[np.float64(0.0),
 np.float64(3.0),
 np.float64(4.0),
 np.float64(3.0),
 np.float64(3.0)]

Convert to tourch tensors

In [20]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Creating Architecture of Autoencoder

In [21]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [28]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.item()*mean_corrector)
      s += 1.
      optimizer.step()

  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.7467633360387202
epoch: 2 loss: 1.0960676798691191
epoch: 3 loss: 1.052949306081064
epoch: 4 loss: 1.0384093400164331
epoch: 5 loss: 1.0307100435154704
epoch: 6 loss: 1.026702756034377
epoch: 7 loss: 1.0236802596372556
epoch: 8 loss: 1.0220758883319065
epoch: 9 loss: 1.0205063846564648
epoch: 10 loss: 1.0198205711362998
epoch: 11 loss: 1.0188725820095834
epoch: 12 loss: 1.0184148509239452
epoch: 13 loss: 1.0177415613066547
epoch: 14 loss: 1.0173704411816133
epoch: 15 loss: 1.0171288924235053
epoch: 16 loss: 1.0168260754295477
epoch: 17 loss: 1.0169181144771342
epoch: 18 loss: 1.016556875943593
epoch: 19 loss: 1.0162048898716567
epoch: 20 loss: 1.0161727562640908
epoch: 21 loss: 1.0160217455197154
epoch: 22 loss: 1.0158308876507185
epoch: 23 loss: 1.016127025001649
epoch: 24 loss: 1.0156592282862973
epoch: 25 loss: 1.0156356608252985
epoch: 26 loss: 1.0155150870168164
epoch: 27 loss: 1.0153647893710187
epoch: 28 loss: 1.014993588329562
epoch: 29 loss: 1.013427530434195


In [30]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.item()*mean_corrector)
    s += 1.
print('Test loss: '+str(test_loss/s))

Test loss: 0.9457375773133804
